# Snake Neural network
Aim of this file is to create a working version of the game Snake
- Then need to be able to have it played by a computer making random decisions
- Once this is completed we will attempt to train a neural network to play it

Run the random model- this will play the game. We only want success cases, so where inputs lead to good outputs
We need:
Training Filter
- input data - what's around the snake head? What action did it take?
- output data - did the snake survive?
Filter the input data by the output data. Where the snake survived, train a model based on the inputs to generate output
- input data - whats around the snakes head
- output data - what action did it take

In [2]:
"""
CURRENT WORK REQUIRED
NEED TO FIX SNAKE VISION - the angle to target isn't quite working
"""

from pygame.locals import *
import pygame
import time
from math import atan2
from math import pi
from math import isnan
from random import randint
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.neural_network import MLPClassifier
import pickle
print('application started..')



class NNtrain:
    actions=np.array([[0,0,0,0,0]])
    events=np.array([[0,0]])
    #vision,action,event
    listOfColumns=["collisionInFront","collisionLeft","collisionRight","AppleAngle","Suggestion","Alive","Success"]
    nnModel=MLPClassifier(hidden_layer_sizes=(6,))
    filename = 'finalized_model.sav'
    snakename='SnakeResults.csv'
    def __init__(self):
        self.actions=self.actions
        self.events=self.events
    def readFile(self):
        X=pd.read_csv(self.snakename)
        y=X[['Alive','Success']]
        X=X[['collisionInFront','collisionLeft','collisionRight','AppleAngle','Suggestion']]
        X=np.array(X)
        y=np.array(y)
        self.actions=X
        self.events=y
    def addNN_data(self,vision,action,event):
        temp=np.concatenate((vision,action),1)
        self.actions=np.concatenate((self.actions,temp),0)
        #self.actions=np.concatenate((self.actions,action),0)
        self.events=np.concatenate((self.events,event),0)
        if self.events[self.events[:,1]==1].shape[0]%10000==0 and event[0,1]==1:
            print(self.events[self.events[:,1]==1].shape[0])
    def returnResults(self):
        temp=np.concatenate((self.actions,self.events),1)
        resultsFile=pd.DataFrame(temp,columns=self.listOfColumns)
        #Please note that this has been commented out as CSV file has already been produced
        resultsFile.to_csv(self.snakename)
    def train(self):
        #****************************************************************
        #Please build training method for model - use Neural Network 
        #You'll need to separate out only the cases where survival occurs - see NN code for details
        #****************************************************************
        
        X=self.actions
        y=self.events
        self.nnModel.fit(X,y)
        
        pickle.dump(self.nnModel, open(self.filename, 'wb'))
        
        print('Model trained')
    def test(self,snake_vision,action):
        l=[]
        #print('Snakevision:')
        #print(snake_vision,action)
        loaded_model = pickle.load(open(self.filename, 'rb'))
        output_size=self.events.shape[1]
        result=-1
        dic={}
        for i in range(2,-1,-1):
            temp=np.concatenate((snake_vision,np.array([[i]])),1)
            this_result=loaded_model.predict(temp)
            l.append([this_result,i])
            if sum(sum(this_result))>1:
                
                return [i,snake_vision]
            else:
                dic[sum(sum(this_result))]=i
        #print('All assessed outputs:')
        #print(l)
        #print('dictionary:')
        #print(dic)
        if len(dic.keys())==1:
            return [randint(0,2),snake_vision]
        else:
            return [dic[max(dic.keys())],(snake_vision,dic.keys())]

#Game class - for collision detection & Game mechanics
class Game:
    def collisionDetect(self,x1,y1,x2,y2,bSize):
        if x1>=x2 and x1<=x2+bSize:
            if y1>=y2 and y1<=y2+bSize:
                return True
        return False

#an Apple to be eaten
class Apple:
    x = 0
    y = 0
    step = 44
 
    def __init__(self,x,y):
        self.x = x * self.step
        self.y = y * self.step
 
    def draw(self, surface, image):
        surface.blit(image,(self.x, self.y)) 

#Defines the Player object - start positions on the board and movements available
class Player:
    x=[]
    y=[]
    speed=1
    step=50
    #length=3
    direction=3
    updateCountMax = 2
    updateCount = 0
    #0-right,1-left,2-up,3-down
    
    def __init__(self,length,width,height):
        self.length=length
        self.width=width
        self.height=height
        for i in range(0,2000):
            self.x.append(200)
            self.y.append(200)
 
       # initial positions, no collision.
        for i in range(1,length):
            self.x[i] += i*50
    #0=right,1=left,2=up,3=down
    def update(self):
        self.updateCount+=1
        #&gt; == >  (Greater than)
        if self.updateCount > self.updateCountMax:
            for i in range(self.length-1,0,-1):
                self.x[i]=self.x[i-1]
                self.y[i]=self.y[i-1]
            #right
            if self.direction==0:
                self.x[0]=self.x[0]+self.step
            #left
            if self.direction==1:
                self.x[0]=self.x[0]-self.step
            #up
            if self.direction==2:
                self.y[0]=self.y[0]-self.step
            #down
            if self.direction==3:
                self.y[0]=self.y[0]+self.step
            self.updateCount=0
    def moveRight(self):
        if self.direction!=1:
            self.direction=0
    def moveLeft(self):
        if self.direction!=0:
            self.direction=1
    def moveUp(self):
        if self.direction!=3:
            self.direction=2
    def moveDown(self):
        if self.direction!=2:
            self.direction=3
    def draw(self, surface, image):
        for i in range(0,self.length):
            surface.blit(image,(self.x[i],self.y[i]))
    def nnPlayer(self):
        nnRandomDirection1=randint(0,2)
        nnRandomDirection2=randint(0,1)
        return nnRandomDirection1,nnRandomDirection2
#Def main Application 
class App:
    windowWidth = 450
    windowHeight = 450
    player = 0
    apple = 0
    vision=np.array([[0,0,0,0]])
    actionTaken=False
    alive=True
    distance_from_apple=sqrt(windowWidth**2+windowHeight**2)
    direction_dic={0:[0,3,2],1:[1,2,3],2:[2,0,1],3:[3,1,0]}
    action=np.array([0])
    score=0
    previous_score=0
    def __init__(self):
        self._running=True
        self._display_surf = None
        self._image_surf = None
        self.player = Player(3,self.windowWidth,self.windowHeight)
        self.apple=Apple(5,5)
        self.game=Game()
        self.nnTrain=NNtrain()
        self.distance_from_apple=self.distance()
    def on_init(self):
        pygame.init()
        self._display_surf = pygame.display.set_mode((self.windowWidth,self.windowHeight), pygame.HWSURFACE)
        pygame.display.set_caption('Pygame pythonspot.com example')
        self._running = True
        self._image_surf = pygame.image.load("pygame.png").convert()
        self._apple_surf = pygame.image.load("apple.png").convert()
        #self._apple_surf = pygame.image.load("apple.png").convert()
    def on_event(self, event):
        if event.type == QUIT:
            self._running = False
 
    def on_loop(self):
        self.player.update()
        self.previous_score=self.score
        if self.game.collisionDetect(self.apple.x,self.apple.y,self.player.x[0],self.player.y[0],50):
            #print(self.apple.x,self.apple.y,self.player.x[0],self.player.y[0],50)
            self.apple.x = randint(2,8) * 44
            self.apple.y = randint(2,8) * 40
            self.player.length += 1
            self.score+=1
            #print('score!')
        if self.player.length>1:
            for i in range(2,self.player.length-1):
                if self.game.collisionDetect(self.player.x[0],self.player.y[0],self.player.x[i], self.player.y[i],50):
                    self.alive=False
                    exit(0)
                
        if self.player.x[0]>=self.windowWidth or self.player.x[0]<=0 or self.player.y[0]>=self.windowHeight or self.player.y[0]<=0:
            #print("You lose! Collision: ")
            #print("x[0] (" + str(self.player.x[0]) + "," + str(self.player.y[0]) + ")")
            #print("x[" + str(i) + "] (" + str(self.player.x[i]) + "," + str(self.player.y[i]) + ")")
            self.alive=False
            exit(0)
            
 
        pass
 
    def on_render(self):
        self._display_surf.fill((0,0,0))
        self.player.draw(self._display_surf, self._image_surf)
        self.apple.draw(self._display_surf, self._apple_surf)
        pygame.display.flip()
 
    def on_cleanup(self):
        pygame.quit()
    def reset(self):
        #print('Dead!')
        self.player.length=3
        self.player.direction=2
        self.player.x=[200 for i in range(0,2000)]
        self.player.y=[200 for i in range(0,2000)]
        self.player.x[1]+=1*50
        self.player.x[2]+=2*50
        self.alive=True
        self.score=0
        self.distance_from_apple=self.distance()
    #Need to create a method by which the snake can see ahead/left/right
    def snake_vision(self):
        #need to establish what's in front, left, right of snake head, and which direction the apple's in
        x=self.player.x
        y=self.player.y
        x0=x[0]
        y0=y[0]
        collisionInFront=0
        collisionLeft=0
        collisionRight=0
        direction=self.player.direction
        if direction==0:
            moveX=50
            moveY=0
            leftX=0
            leftY=-50
            opp=self.apple.y-y0
            adj=self.apple.x-x0
        elif direction==1:
            moveX=-50
            moveY=0
            leftX=0
            leftY=50
            opp=-(self.apple.y-y0)
            adj=-(self.apple.x-x0)
        elif direction==2:
            moveX=0
            moveY=-50
            leftX=-50
            leftY=0
            opp=self.apple.x-x0
            adj=-(self.apple.y-y0)
        else:
            moveX=0
            moveY=50
            leftX=50
            leftY=0
            opp=-(self.apple.x-x0)
            adj=(self.apple.y-y0)
        #detect what's infront
        if self.player.length>1:
            for i in range(2,self.player.length-1):
                if self.game.collisionDetect(x0+moveX,y0+moveY,x[i],y[i],50):
                    collisionInFront=1
            for i in range(1,self.player.length-1):
                if self.game.collisionDetect(x0+leftX,y0+leftY,x[i],y[i],50):
                    collisionLeft=1
            for i in range(1,self.player.length-1):
                if self.game.collisionDetect(x0-leftX,y0-leftY,x[i],y[i],50):
                    collisionRight=1
        if x0+moveX>=self.windowWidth or x0+moveX<=0 or y0+moveY>=self.windowHeight or y0+moveY<=0:
            collisionInFront=1
        #detect what's next to left
        if x0+leftX>=self.windowWidth or x0+leftX<=0 or y0+leftY>=self.windowHeight or y0+leftY<=0:
            collisionLeft=1
        #detect what's next to right
        if x0-leftX>=self.windowWidth or x0-leftX<=0 or y0-leftY>=self.windowHeight or y0-leftY<=0:
            collisionRight=1
        #detect what angle the apple is from head - measured in radians - right of snake head screen up - or left
        #right
        """
        snake_direction=np.array([self.player.x[0]-self.player.x[1],self.player.y[0]-self.player.y[1]])
        food_direction=np.array([self.apple.x-self.player.x[0],self.apple.y-self.player.y[0]])
        apple_angle=float(self.get_angle(snake_direction,food_direction))
        #print(apple_angle)
        if isnan(apple_angle):
            apple_angle=0
        """
        #current amendment to code
        apple_angle=atan2(opp,adj)

        return np.array([[collisionInFront,collisionLeft,collisionRight,apple_angle]])
    def distance(self):
        return np.linalg.norm(np.array([self.apple.y,self.apple.x])-np.array([self.player.y[0],self.player.x[0]]))
    def normalize_vector(self, vector):
        return vector / np.linalg.norm(vector)
    def get_angle(self, a, b):
        a = self.normalize_vector(a)
        b = self.normalize_vector(b)
        return atan2(a[0] * b[1] - a[1] * b[0], a[0] * b[0] + a[1] * b[1]) / pi
    def successful_move(self):
        if self.distance_from_apple>self.distance() or self.score>self.previous_score:
            return 1
        elif self.distance_from_apple==self.distance():
            return -1
        else:
            return 0
    def on_execute(self,train_test):
        if self.on_init() == False:
            self._running = False
        self.nnTrain.readFile()
        while(self._running):
            pygame.event.pump()
            keys = pygame.key.get_pressed()
            self.vision=self.snake_vision()
            self.distance_from_apple=self.distance()
#*******************************************************************
#Need to amend this section so that there is input from the model once it's trained
#*******************************************************************
            
            if train_test=="train":
                suggestion,decision=self.player.nnPlayer()
                computerPlayer=self.direction_dic[self.player.direction][suggestion]
                deathrslt=""
            else:
                suggestion,decision=self.player.nnPlayer()
                suggestion=np.array([[suggestion]])
                computerPlayer=self.direction_dic[self.player.direction][(self.nnTrain.test(self.vision,suggestion))[0]]
                deathrslt=(self.nnTrain.test(self.vision,suggestion))
                #print(computerPlayer)
            

            self.player.direction=computerPlayer
            suggestion=np.array([[suggestion]])
            
            if (keys[K_RIGHT]) or computerPlayer==0:
                self.player.moveRight()
                self.action=np.array([[0]])
                self.actionTaken=True
 
            if (keys[K_LEFT]) or computerPlayer==1:
                self.player.moveLeft()
                self.action=np.array([[1]])
                self.actionTaken=True
 
            if (keys[K_UP]) or computerPlayer==2:
                self.player.moveUp()
                self.action=np.array([[2]])
                self.actionTaken=True
 
            if (keys[K_DOWN]) or computerPlayer==3:
                self.player.moveDown()
                self.action=np.array([[3]])
                self.actionTaken=True
 
            if (keys[K_ESCAPE]):
                self._running = False
                if train_test=="train":
                    self.nnTrain.returnResults()
                    self.nnTrain.train()
            
            self.on_loop()
            self.on_render()
            if self.successful_move()!=-1 and train_test!="train" and train_test!="test":
                print("success?: " + str(self.successful_move()))
                print("previous distance: " + str(self.distance_from_apple))
                print("New distance: " + str(self.distance()))
                print('Action taken: ' + str(self.action))
            if self.alive==True and train_test=="train":
                if self.successful_move()==1:
                    self.nnTrain.addNN_data(self.vision,suggestion,np.array([[1,1]]))
                elif self.successful_move()==0:
                    self.nnTrain.addNN_data(self.vision,suggestion,np.array([[1,0]]))
            elif train_test=="train" and self.alive==False:
                self.nnTrain.addNN_data(self.vision,suggestion,np.array([[0,0]]))
                self.reset()
                
            elif self.alive==False:
                #experiment with training network based on behaviour data
                #self.nnTrain.addNN_data(self.vision,suggestion,np.array([[0,0]]))
                print('Death comes for ye!')
                print(deathrslt)
                print(self.apple.x,self.apple.y,self.player.x[0],self.player.y[0])
                self.reset()
                
                
            self.actionTaken=False
            
            #Add a timer!
            if train_test=="train":
                time.sleep (0.01/1000.0)
            else:
                time.sleep (100/1000.0)
            
        self.on_cleanup()
 

if __name__ == "__main__" :
    theApp = App()
    theApp.on_execute("test")


application started..


In [1]:
#we can use this to train the data if you want to mess around with it

from pygame.locals import *
import pygame
import time
from math import atan2
from math import pi
from math import isnan
from random import randint
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.neural_network import MLPClassifier
import pickle
print('application started..')
X=pd.read_csv('SnakeResults.csv')
y=X[['Alive','Success']]
X=X[['collisionInFront','collisionLeft','collisionRight','AppleAngle','Suggestion']]
X=np.array(X)
y=np.array(y)
print(X[:3,:])
nnModel=MLPClassifier(hidden_layer_sizes=(6,))
nnModel.fit(X[1:,:],y[1:,:])
filename = 'finalized_model.sav'
pickle.dump(nnModel, open(filename, 'wb'))
print('Model trained')

application started..
[[ 0.          0.          0.          0.          0.        ]
 [ 0.          1.          0.         -0.78539816  0.        ]
 [ 0.          0.          0.         -2.58299334  2.        ]]
Model trained
